In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
names = open('data/names.txt', 'r').read().splitlines()
names[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
letters = sorted(list(set(''.join(names))))
SoS = '<.>'
letters = [SoS] + letters 
letter2index = {l:i for i, l in enumerate(letters)}
index2letter = {i:l for l, i in letter2index.items()}
print(index2letter)

{0: '<.>', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [5]:
vocab_size = len(letter2index)
vocab_size

27

In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in list(w) + [SoS]:
      ix = letter2index[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

Xtr,  Ytr  = build_dataset(names[:n1])     # 80%
Xdev, Ydev = build_dataset(names[n1:n2])   # 10%
Xte,  Yte  = build_dataset(names[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [8]:
embedding_size = 10
hidden_size = 64

g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, embedding_size), generator=g)
W1 = torch.randn((block_size * embedding_size, hidden_size), generator=g)* (5/3)/((embedding_size * block_size)**0.5)
b1 = torch.randn((1, hidden_size), generator=g) * 0.1
W2 = torch.randn((hidden_size, vocab_size), generator=g) * 0.1
b2 = torch.randn((1, vocab_size), generator=g) * 0.1

bngain = torch.randn((1, hidden_size))*0.1 + 1.0
bnbias = torch.randn((1, hidden_size))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [9]:
def random_batch(X, Y, batch_size):
    batch_idx = torch.randint(0, X.size(0), (batch_size, ))
    return X[batch_idx], Y[batch_idx]

In [10]:
Xb, Yb = random_batch(Xtr, Ytr, 32)

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time
n = Xb.size(0)

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.4836, grad_fn=<NegBackward0>)

In [12]:
Yb

tensor([13, 12,  0,  0, 20,  0,  4, 12,  5, 15,  4,  0, 11,  1,  9, 12,  1, 24,
        14,  0, 15, 14, 24,  0,  5,  5,  5,  9,  1, 20,  0,  5])

In [13]:
logprobs[range(n), Yb]

tensor([-3.3489, -3.4355, -3.7568, -2.9241, -2.9333, -3.2417, -4.6258, -2.6613,
        -3.8851, -2.8886, -4.0868, -3.0793, -3.5364, -3.5937, -3.2925, -3.7106,
        -4.4511, -4.0723, -3.8339, -2.7458, -4.5336, -2.8755, -4.0198, -2.8856,
        -2.9595, -3.6142, -3.5466, -4.4321, -3.0003, -3.1061, -3.0801, -3.3188],
       grad_fn=<IndexBackward0>)

In [14]:
logprobs.shape

torch.Size([32, 27])

In [15]:
counts.shape, counts_sum_inv.shape, counts.sum(dim=1, keepdim=True).shape

(torch.Size([32, 27]), torch.Size([32, 1]), torch.Size([32, 1]))

In [16]:
logits.shape, logit_maxes.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [17]:
logits[range(n), logits.argmax(dim=1)]

tensor([1.1554, 1.2180, 1.3205, 0.7188, 1.2426, 0.8789, 1.0674, 0.7745, 0.7962,
        0.4934, 1.6195, 1.1961, 1.5247, 1.2585, 0.8596, 1.0127, 1.2180, 1.2180,
        0.8462, 0.9565, 1.3203, 1.1026, 1.3401, 0.9577, 1.7787, 1.1946, 1.1569,
        1.1874, 0.9350, 1.3362, 0.9478, 0.9576], grad_fn=<IndexBackward0>)

In [18]:
logits.max(1, keepdim=True).indices

tensor([[19],
        [22],
        [15],
        [15],
        [15],
        [21],
        [ 6],
        [12],
        [22],
        [20],
        [ 3],
        [23],
        [13],
        [21],
        [ 6],
        [19],
        [22],
        [22],
        [21],
        [20],
        [24],
        [23],
        [13],
        [19],
        [16],
        [22],
        [13],
        [19],
        [21],
        [ 3],
        [ 1],
        [ 2]])

In [19]:
logits[range(n), logits.max(1, keepdim=True).indices.view(-1)]

tensor([1.1554, 1.2180, 1.3205, 0.7188, 1.2426, 0.8789, 1.0674, 0.7745, 0.7962,
        0.4934, 1.6195, 1.1961, 1.5247, 1.2585, 0.8596, 1.0127, 1.2180, 1.2180,
        0.8462, 0.9565, 1.3203, 1.1026, 1.3401, 0.9577, 1.7787, 1.1946, 1.1569,
        1.1874, 0.9350, 1.3362, 0.9478, 0.9576], grad_fn=<IndexBackward0>)

In [20]:
F.one_hot(logits.max(1, keepdim=True).indices.view(-1), num_classes=logits.size(1)).shape

torch.Size([32, 27])

In [21]:
logits.shape, b2.shape

(torch.Size([32, 27]), torch.Size([1, 27]))

In [22]:
h.shape, W2.shape, logits.shape

(torch.Size([32, 64]), torch.Size([64, 27]), torch.Size([32, 27]))

In [23]:
hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape


(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [24]:
bndiff.shape, bnvar_inv.shape


(torch.Size([32, 64]), torch.Size([1, 64]))

In [38]:
# emb = C[Xb]
print(emb.shape, C.shape, Xb.shape, )
Xb[:4]

torch.Size([32, 3, 10]) torch.Size([27, 10]) torch.Size([32, 3])


tensor([[12,  9,  1],
        [ 0,  0,  0],
        [20, 15, 14],
        [11, 23, 21]])

In [40]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)
dprobs = (1/probs) * dlogprobs
cmp('dprobs', dprobs, probs)
dcounts_sum_inv = (counts * dprobs).sum(dim=1, keepdim=True)
cmp('dcounts_sum_inv', dcounts_sum_inv, counts_sum_inv)
dcounts = counts_sum_inv * dprobs
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
cmp('dcounts_sum', dcounts_sum, counts_sum)
dcounts += torch.ones_like(counts) * dcounts_sum
cmp('dcounts', dcounts, counts)
dnorm_logits = counts * dcounts
cmp('dnorm_logits', dnorm_logits, norm_logits)
dlogit_maxes = -dnorm_logits.sum(dim=1, keepdim=True)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)
dlogits = dnorm_logits
dlogits += F.one_hot(logits.max(1, keepdim=True).indices.view(-1), num_classes=logits.size(1)) * dlogit_maxes
cmp('dlogits', dlogits, logits)
db2 = dlogits.sum(dim=0, keepdim=True)
cmp('db2', db2, b2)
dh = dlogits @ W2.T
cmp('dh', dh, h)
dW2 = h.T @ dlogits
cmp('dW2', dW2, W2)
dhpreact = (1 - h**2) * dh  # h = torch.tanh(hpreact) tanh(x) = (e^(2x) - 1) / (e^(2x) - 1), dtanh = 1 - tanh^2
cmp('dhpreact', dhpreact, hpreact)
dbnbias = dhpreact.sum(dim=0, keepdim=True)
cmp("dbnbias", dbnbias, bnbias)
dbngain = (dhpreact * bnraw).sum(dim=0, keepdim=True)
cmp("dbngain", dbngain, bngain)
dbnraw = dhpreact * bngain
cmp("dbnraw", dbnraw, bnraw)
dbnvar_inv = (dbnraw * bndiff).sum(dim=0, keepdim=True)
cmp("dbnvar_inv", dbnvar_inv, bnvar_inv)
dbndiff = dbnraw * bnvar_inv
dbnvar = -0.5 * dbnvar_inv * (bnvar + 1e-5)**-1.5 #bnvar_inv = (bnvar + 1e-5)**-0.5
cmp("dbnvar", dbnvar, bnvar)
dbndiff2 = 1/(n-1) * torch.ones_like(bndiff2)  * dbnvar # bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
cmp("dbndiff2", dbndiff2, bndiff2)
#bndiff2 = bndiff**2
dbndiff += 2 * bndiff * dbndiff2
cmp("dbndiff", dbndiff, bndiff)
# bndiff = hprebn - bnmeani
dhprebn = dbndiff
dbnmeani = -dbndiff.sum(dim=0, keepdim=True)
cmp("dbnmeani", dbnmeani, bnmeani)
# bnmeani = 1/n * hprebn.sum(0, keepdim=True)
dhprebn += 1 / n * torch.ones_like(hprebn) * dbnmeani
cmp("dhprebn", dhprebn, hprebn)
# hprebn = embcat @ W1 + b1
db1 = dhprebn.sum(dim=0, keepdim=True)
cmp('db1', db1, b1)
dembcat = dhprebn @ W1.T
cmp('dembcat', dembcat, embcat)
dW1 = embcat.T @ dhprebn
cmp('dW1', dW1, W1)
# embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
demb = dembcat.reshape(emb.shape)
cmp('demb', demb, emb)
# emb = C[Xb] # embed the characters into vectors
dC = torch.zeros_like(C)
for sample_i in range(Xb.size(0)):
    for char_i in range(Xb.size(1)):
        ix = Xb[sample_i, char_i]
        for emb_i in range(dC.size(1)):
            dC[ix, emb_i] += demb[sample_i, char_i, emb_i]
cmp('dC', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum_inv | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0
dcounts         | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
dlogits         | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dW2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: True  | approximate: True  | maxdiff: 0.0
dbnbias         | exact: True  | approximate: True  | maxdiff: 0.0
dbngain         | exact: True  | approximate: True  | maxdiff: 0.0
dbnraw          | exact: True  | approximate: True  | maxdiff: